In [1]:
from stonks.data.data_reader import MktDataReader


from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions 

In [2]:
tickers = [
    'AAPL', # Apple -> tech
    'TSLA', # Tesla -> tech
    'GOOGL', # Google -> tech
    'AMZN', # Amazon -> consumer/tech
    'MA', # MasterCard -> finance
    'V', # Visa -> finance
    'MELI', # MercadoLibre -> consumer
    'NVDA', # Nvidia -> chips
    'ASML', # ASML -> chips
    'FB' # facebook -> social media
]

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2008-01-01'
end_date = '2021-04-05'

dr = MktDataReader(start_date, end_date, 'yahoo', tickers)

[*********************100%***********************]  10 of 10 completed


In [3]:
# impute missing data
dr.impute_missing_data()
# fetch market caps
dr.fetch_market_caps()

In [6]:
# get mean and cover matrices
mu = mean_historical_return(dr.df_stocks_bkfilled)
S = CovarianceShrinkage(dr.df_stocks_bkfilled).ledoit_wolf()

# with l2 regularization to not have 0s in the weights

ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.5)
w = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.txt")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.11133), ('AMZN', 0.14094), ('ASML', 0.12624), ('FB', 0.07028), ('GOOGL', 0.01964), ('MA', 0.09401), ('MELI', 0.0556), ('NVDA', 0.05144), ('TSLA', 0.22766), ('V', 0.10285)])
Expected annual return: 30.0%
Annual volatility: 26.4%
Sharpe Ratio: 1.06


/home/charx/python-virtual-environments/stonks-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimization problem so additional objectives may not work as expected."


(0.2998621747002599, 0.2636093550703512, 1.0616549425022157)